# reaching_go_spout_bar_nov22

```bash
jupyter nbconvert "D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical\nb20221201_073500_reaching_go_spout_bar_nov22.ipynb" --to="python" --output-dir="D:\OneDrive - Nexus365\Private_Dropbox\Projects\trialexp\notebooks\noncanonical" --output="nb20221201_073500_reaching_go_spout_bar_nov22"
```

Quick analysis of instrumental reaching

In [1]:
# allow for automatic reloading of classes and function when updating the code
%load_ext autoreload
%autoreload 2 

# Import Session and Experiment class with helper functions
from trialexp.process.data_import import *



### Variables

In [2]:
import pandas as pd

trial_window = [-2000, 6000] # in ms

# time limit around trigger to perform an event
# determine successful trials
timelim = [0, 2000] # in ms

# Digital channel nb of the pyphotometry device
# on which rsync signal is sent (from pycontrol device)
rsync_chan = 2

basefolder, _ = os.path.split(os.path.split(os.getcwd())[0])

# These must be absolute paths
# use this to use within package tasks files (in params)
tasksfile = os.path.join(basefolder,'params\\tasks_params.csv')
# use this to put a local full path
#tasksfile = -r'C:/.../tasks_params.csv' 

# photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\test_folder\photometry'
photometry_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\kms_pyphotometry'
video_dir = r'\\ettin\Magill_Lab\Julien\Data\head-fixed\videos'

In [3]:
tasks = pd.read_csv(tasksfile, usecols=[1, 2, 3, 4], index_col=False)
tasks


,task,triggers,events,conditions
0,lick_go_nogo,go; nogo,bar; bar_off; spout,free_reward
1,lick_go_nogo_unconditionned,go; nogo,bar; bar_off; spout,free_reward
2,lick_go_nogo_unconditionned_opto,go; nogo,bar; bar_off; spout,free_reward
3,reaching_go_nogo,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
4,reaching_go_nogo_jc,CS_plus; CS_minus,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
5,reaching_go_nogo_opto_continuous,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset; s_go_sham; s_go_...
6,reaching_go_nogo_opto_sinusoid,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
7,reaching_go_nogo_opto_sinusoid_spout,CS_Go; CS_NoGo,bar; bar_off; spout,s_nogo_sham; s_nogo_cs_onset_2; s_nogo_cs_onse...
8,reaching_go_nogo_reversal,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...
9,reaching_go_nogo_reversal_incentive,CS_Go; CS_NoGo,bar; bar_off; spout,error bar_off; reward bar_off; error free; rew...


In [4]:
photo_root_dir = 'T:\\Data\\head-fixed\\pyphotometry\\data'
pycontrol_root_dir = 'T:\\Data\\head-fixed\\pycontrol'

root_folders = [photo_root_dir, pycontrol_root_dir]
horizontal_folder_pycontrol = 'T:\\Data\\head-fixed\\test_folder\\pycontrol'
horizontal_folder_photometry = 'T:\\Data\\head-fixed\\test_folder\\photometry'

copy_files_to_horizontal_folders(
    root_folders, horizontal_folder_pycontrol, horizontal_folder_photometry)


### Create an experiment object


In [5]:
# Folder of a full experimental batch, all animals included

# Enter absolute path like this
# pycontrol_files_path = r'T:\Data\head-fixed\test_folder\pycontrol'

# or this if you want to use data from the sample_data folder within the package
pycontrol_files_path = os.path.join(basefolder, 'sample_data/pycontrol')
pycontrol_files_path = r'T:\Data\head-fixed\kms_pycontrol'

# Load all raw text sessions in the indicated folder or a sessions.pkl file
# if already existing in folder_path
exp_cohort = Experiment(pycontrol_files_path, update = True) #TODO

# Only use if the Experiment cohort as been processed by trials before
# TODO: assess whether this can be removed or not
exp_cohort.by_trial = True


Saved sessions loaded from: sessions.pkl


## Select sessions

In [6]:
import datetime
ss = exp_cohort.sessions

ss_ = [this_ss for this_ss in ss 
       # [313, 314, 315, 316, 317, 318]
       if (this_ss.subject_ID in [ 314, 316])
    and ( this_ss.experiment_name == 'reaching_go_spout_bar_nov22')
    and (this_ss.datetime.date() == datetime.date(2022,11,30))]
ss_


In [7]:
exp_cohort.sessions = ss_

In [8]:
ss_


In [9]:
ss_[0].datetime.date()


datetime.date(2022, 11, 30)

# SLOW 3m

In [10]:

# # Process the whole experimental folder by trials

# exp_cohort.process_exp_by_trial(
#     trial_window, timelim, tasksfile, blank_spurious_event='spout', blank_timelim=[0, 65])
#     # not working

# # Find if there is a matching photometry file and if it can be used:
# # rsync synchronization pulses matching between behaviour and photometry

# # Find if there is a matching photometry file:
# exp_cohort.match_sessions_to_files(photometry_dir, ext='ppd')

# # rsync synchronization pulses matching between behaviour and photometry
# exp_cohort.sync_photometry_files(2)

# # Find matching videos
# exp_cohort.match_sessions_to_files(video_dir, ext='mp4')

# # FInd matching DeepLabCut outputs files
# exp_cohort.match_sessions_to_files(video_dir, ext='h5', verbose=True)


# # exp_cohort.save()


In [11]:
exp_cohort.subject_IDs


[314, 316]

In [12]:
# Many combinations possible
conditions_dict0 = {'trigger': 'hold_for_water', 'valid': True}


# Aggregate all condition dictionaries in a list
condition_list = [conditions_dict0]
# Aliases for conditions
cond_aliases = [
    'any_trial',
]

# Groups as a list of lists
groups = None

# right_handed = [281]
# groups = [[280, 282, 299, 300, 301],\
#     [284, 285, 296, 297, 306, 307]]
# Window to exctract (in ms)


In [13]:
exp_cohort.sessions[0].times.keys()

dict_keys(['hold_for_water', 'waiting_for_spout', 'waiting_for_bar', 'busy_win', 'break_after_water', 'break_after_no_water', 'spout_off', 'US_end_timer', 'action_win_timer', 'block_timer', 'button_press', 'bar_off', 'US_delay_timer', 'CS_offset_timer', 'busy_win_timer', 'waiting_for_bar_timer', 'spout_ana', 'sound_off_timer', 'hold_timer', 'spout_ana_off', 'free_water_timer', 'rsync', 'bar', 'spout', 'button_release'])

# Session plot 

I realised that this plot can never tell if a water drop was triggered by bar_off or spout.


In [14]:
exp_cohort.sessions[0].print_lines[0:30]

['17104 water_on',
 '17104 water by spout',
 '18906 Timestamp:15690, this trial > Success:True, trial_type:spout, 1 spout touchces',
 '34975 water_on',
 '34975 water by spout',
 '36777 Timestamp:34034, this trial > Success:True, trial_type:spout, 1 spout touchces',
 '54246 Timestamp:51244, this trial > Success:False, trial_type:spout, 0 spout touchces',
 '77458 Timestamp:74457, this trial > Success:False, trial_type:spout, 0 spout touchces',
 '95773 Timestamp:92771, this trial > Success:False, trial_type:spout, 0 spout touchces',
 '113501 Timestamp:110499, this trial > Success:False, trial_type:spout, 0 spout touchces',
 '129274 water_on',
 '129274 water by spout',
 '130122 water_on',
 '130122 water by spout',
 '131076 Timestamp:128316, this trial > Success:True, trial_type:spout, 4 spout touchces',
 '148706 water_on',
 '148706 water by spout',
 '149170 water_on',
 '149170 water by spout',
 '150507 Timestamp:145867, this trial > Success:True, trial_type:spout, 4 spout touchces',
 '1681

In [15]:
import re

re.match('abc ','abc de')

expr = '^\d+(?= ' + '.?Timestamp' + ')'
a = [re.match(expr, L) for L in exp_cohort.sessions[0].print_lines if re.match(expr , L) is not None]
int(a[0].group(0))

18906

In [16]:
for ss in exp_cohort.sessions:
    smrxname = re.sub('\.txt', f'_{ss.task_name}.smrx', ss.file_name)
    print(smrxname)


JC314L-2022-11-30-153050_reaching_go_spout_bar_nov22.smrx
JC316L-2022-11-30-163444_reaching_go_spout_bar_nov22.smrx



# export

In [20]:
keys = [
        'button_press', 'bar', 'bar_off', 'spout', 'US_delay_timer', 'CS_offset_timer']
state_def = [{'name': 'hold_for_water', 'onset': 'hold_for_water', 'offset': 'waiting_for_spout'},
                    {'name': 'waiting_for_spout', 'onset': 'waiting_for_spout',
                    'offset': 'busy_win'},
                    {'name': 'busy_win', 'onset': 'busy_win',
                        'offset': 'break_after_water'},
                    {'name': 'break_after_water', 'onset': 'break_after_water',    'offset': 'waiting_for_bar'},
                    {'name': 'break_after_no_water',       'onset': 'break_after_no_water', 'offset': 'waiting_for_bar'}]
summary_df = pd.DataFrame()

for ss in exp_cohort.sessions:
    smrxname = re.sub('\.txt', f'_{ss.task_name}.smrx', ss.file_name)
    print(smrxname)


    bw = ss.times['busy_win']
    sp = ss.times['spout']

    x_spout = [this_bw for this_bw in bw for spouts in sp if (
        spouts < this_bw) and (this_bw - spouts < 100)]

    x_bar = [this_bw for this_bw in bw if not any(
        [(spouts < this_bw) and (this_bw - spouts < 100) for spouts in sp])]
        
    event_ms = [{
        'name': 'triggered by spout',
        'time_ms': x_spout
    },
        {
            'name': 'triggered by bar_off',
            'time_ms': x_bar
    }
    ]

    if re.search('11\-23',ss.file_name): #adapt to a bug 
        state_def[-1]['offset'] = 'wating_for_bar'
    else:
        state_def[-1]['offset'] = 'waiting_for_bar'


    ss.plot_session(
        keys, state_def, export_son=True, event_ms=event_ms, son_filename= smrxname)

    summary_df = summary_df.append({
        'file':ss.file_name,
        'task':ss.task_name,
        'triggered_by_spout': len(x_spout),
        'triggered_by_bar_off': len(x_bar),
        'reaching_trials': len(bw),
        'trials': len(ss.times['hold_for_water'])},
        ignore_index=True)


JC314L-2022-11-30-153050_reaching_go_spout_bar_nov22.smrx


saved JC314L-2022-11-30-153050_reaching_go_spout_bar_nov22.smrx
JC316L-2022-11-30-163444_reaching_go_spout_bar_nov22.smrx


C:\Users\phar0528\AppData\Local\Temp\ipykernel_29652\314481084.py:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



saved JC316L-2022-11-30-163444_reaching_go_spout_bar_nov22.smrx


C:\Users\phar0528\AppData\Local\Temp\ipykernel_29652\314481084.py:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## trouble shooting for 314

error is in event channel

which channel?
ReadEvents seems working for the first 10 divides at least

In [17]:
keys = [
        'button_press', 'bar', 'bar_off', 'spout', 'US_delay_timer', 'CS_offset_timer']
state_def = [{'name': 'hold_for_water', 'onset': 'hold_for_water', 'offset': 'waiting_for_spout'},
                    {'name': 'waiting_for_spout', 'onset': 'waiting_for_spout',
                    'offset': 'busy_win'},
                    {'name': 'busy_win', 'onset': 'busy_win',
                        'offset': 'break_after_water'},
                    {'name': 'break_after_water', 'onset': 'break_after_water',    'offset': 'waiting_for_bar'},
                    {'name': 'break_after_no_water',       'onset': 'break_after_no_water', 'offset': 'waiting_for_bar'}]
summary_df = pd.DataFrame()

for ss in [exp_cohort.sessions[0]]:
    smrxname = re.sub('\.txt', f'_{ss.task_name}.smrx', ss.file_name)
    print(smrxname)


    bw = ss.times['busy_win']
    sp = ss.times['spout']

    x_spout = [this_bw for this_bw in bw for spouts in sp if (
        spouts < this_bw) and (this_bw - spouts < 100)]

    x_bar = [this_bw for this_bw in bw if not any(
        [(spouts < this_bw) and (this_bw - spouts < 100) for spouts in sp])]
        
    event_ms = [{
        'name': 'triggered by spout',
        'time_ms': x_spout
    },
        {
            'name': 'triggered by bar_off',
            'time_ms': x_bar
    }
    ]

    if re.search('11\-23',ss.file_name): #adapt to a bug 
        state_def[-1]['offset'] = 'wating_for_bar'
    else:
        state_def[-1]['offset'] = 'waiting_for_bar'


    ss.plot_session(
        keys, state_def, export_son=True, event_ms=event_ms, son_filename= smrxname, verbose=True)

    summary_df = summary_df.append({
        'file':ss.file_name,
        'task':ss.task_name,
        'triggered_by_spout': len(x_spout),
        'triggered_by_bar_off': len(x_bar),
        'reaching_trials': len(bw),
        'trials': len(ss.times['hold_for_water'])},
        ignore_index=True)


JC314L-2022-11-30-153050_reaching_go_spout_bar_nov22.smrx
1, button_press:
[16830000, 34702000, 37206000, 37380000, 129000000, 429733000]
2, bar:
[10208000, 10375000, 11835000, 12571000, 12680000, 12838000, 13233000, 17030000, 17123000, 17260000]
3, bar_off:
[10178000, 10323000, 11762000, 12545000, 12634000, 12785000, 13188000, 16964000, 17046000, 17138000]
4, spout:
[16903000, 17176000, 34774000, 35047000, 37278000, 37453000, 37691000, 37814000, 37970000, 38246000]
5, US_delay_timer:
[17104000, 34975000, 129274000, 130122000, 148706000, 149170000, 185585000, 186233000, 353072000, 353651000]
6, CS_offset_timer:
[2000000]
7, triggered by spout:
[16904000, 34775000, 129074000, 185385000, 429807000, 548149000, 642140000, 740352000, 757689000, 774782000]
8, triggered by bar_off:
[148505000, 352871000, 373967000, 509556000, 566417000, 604439000, 721684000, 839192000, 922377000, 958807000]
9, print lines:
error in print
10, hold_for_water:
[<sonpy.DigMark> with Tick = 15490000 and Codes = [1

saved JC314L-2022-11-30-153050_reaching_go_spout_bar_nov22.smrx


C:\Users\phar0528\AppData\Local\Temp\ipykernel_34176\2935295093.py:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

